# MULTICAST

Apartado 1

Esta conexión multicast se realiza en un grupo de multidifusión, es por ello que no tenemos un servidor, si no otro cliente, que mientras que estén en el mismo puerto y el mismo grupo tendrán conexión entre ellos. La funcionalidad es muy simple, estarán conectados hasta que el cliente escriba Adios, en ese momento todos los participantes saldrán del grupo

In [ ]:
import socket
import struct

MCAST_GRP = '224.1.1.1'
MCAST_PORT = 5007

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

#Usar '' para esuchar y escribir en el socket
#Usar MCAST_GRP para escuchar solo en el socket
sock.bind(('', MCAST_PORT))
sock.sendto(str.encode("Hello this is user 1"), (MCAST_GRP, MCAST_PORT))

mreq = struct.pack("4sl", socket.inet_aton(MCAST_GRP), socket.INADDR_ANY)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_ADD_MEMBERSHIP, mreq)

#Establecer el ttl
ttl = struct.pack('b',1)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_MULTICAST_TTL, ttl)

msg = ""
while str(msg) != "Adios":
    mensaje_tx = str.encode(input('Introduzca el mensaje a enviar: '))
    sock.sendto(mensaje_tx, (MCAST_GRP, MCAST_PORT))
    msg = sock.recv(1024).decode()
    print("RX: ", msg)
    
print("Cliente abandona el grupo")                       
sock.close()

Introduzca el mensaje a enviar: asdadasddad
RX:  asdadasddad
Introduzca el mensaje a enviar: asdasddasds
RX:  Hello this is user 2
Introduzca el mensaje a enviar: asdada
RX:  asdasddasds
Introduzca el mensaje a enviar: asdasdasd
RX:  asdada
Introduzca el mensaje a enviar: asdadsa
RX:  asdasdasd
Introduzca el mensaje a enviar: holaaaaa
RX:  asdadsa
Introduzca el mensaje a enviar: asdasdsad
RX:  holaaaaa


Apartado 2

En esta ocasión lo que hemos querido implementar es una especie de chat. Para ellos nos hemos ayudado de pickle para empaquetar tanto el mensaje como el identificador del participante y poder mandarlo al grupo. Nos hemos ayudado de una función de tiempo para que la conexión y escucha se realice correctamente. El grupo estará activo hasta que el usuario escriba quit, en ese momento cerrará la conexión.

In [ ]:
import socket
import struct
import pickle
import threading

MCAST_GRP = '224.1.1.1'
MCAST_PORT = 5007

# identificador del participante
user_id = 1

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

#Usar '' para esuchar y escribir en el socket
#Usar MCAST_GRP para escuchar solo en el socket
sock.bind(('', MCAST_PORT))
mensaje_bienvenida = 'Hello this is user 1'
bytes_bienvenida = pickle.dumps([mensaje_bienvenida, user_id])
sock.sendto(bytes_bienvenida, (MCAST_GRP, MCAST_PORT))

mreq = struct.pack("4sl", socket.inet_aton(MCAST_GRP), socket.INADDR_ANY)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_ADD_MEMBERSHIP, mreq)

#Establecer el ttl
ttl = struct.pack('b',1)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_MULTICAST_TTL, ttl)

def time_function(name):
    global stop
    while stop == False:
        msg = sock.recv(1024)
        mensaje = pickle.loads(msg)
        print("Thread User " + str(mensaje[1]) + ": " + str(mensaje[0]))
        
    print("Cliente abandona el grupo")
    sock.close()
    
stop = False
reader = threading.Thread(target=time_function, args=(user_id,))
reader.start()

while stop == False:
    mensaje_tx = input('Introduzca el mensaje a enviar: ')
    bytes_tx = pickle.dumps([mensaje_tx, user_id])
    sock.sendto(bytes_tx, (MCAST_GRP, MCAST_PORT))
    if mensaje_tx == "quit":
        stop = True
        break

sock.close()

Thread User 2: Hello this is user 2
Thread User 2: asidjasdja
Introduzca el mensaje a enviar: asdasdad
Thread User 1: asdasdad
Introduzca el mensaje a enviar: asd
Thread User 1: asd
Introduzca el mensaje a enviar: 
Thread User 1: 
Introduzca el mensaje a enviar: asd
Thread User 1: asd
Introduzca el mensaje a enviar: 
Thread User 1: 
Introduzca el mensaje a enviar: asd
Thread User 1: asd
Introduzca el mensaje a enviar: asd
Thread User 1: asd
Introduzca el mensaje a enviar: asd
Thread User 1: asd
Thread User 2: asdd
Thread User 2: asd
Thread User 2: asd
Thread User 2: asd
Thread User 2: asd
Introduzca el mensaje a enviar: asd
Thread User 1: asd
